# Plotting Vietnam Development Map



## Table of contents
1. [The Aquaculture Production of Vietnam from 2013 to 2018 by Provinces](#1)<br>
    1.1 [Data](#11)<br>
    1.2 [Choropleth map using GeoJSON](#12)<br>
    1.3 [Animated figures with Plotly Express](#13)<br>
2. [Proportion of poor households by region in Vietnam from 1998 to 2016 ](#2)<br>
    2.1 [Data](#21)<br>
    2.2 [Choropleth map using GeoJSON](#22)<br>
    2.3 [Animated figures with Plotly Express](#23)<br>
5. [Important Notes](#333)<br>
5. [References](#666)<br>

In [1]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

# Plot in browser (recommended)
import plotly.io as pio
pio.renderers.default = 'browser'

# Make Plotly work in your Jupyter Notebook
# import cufflinks as cf
# import chart_studio.plotly as py
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode(connected=True)
# # Use Plotly locally
# cf.go_offline()

<img src="figures/vietnam-map-with-states.jpg" alt="drawing" style="width:300px;"/>

## 1. The Aquaculture Production of Vietnam from 2013 to 2018 by Provinces <a id="1"></a>
Published by: Open Development Vietnam The data provides information on Vietnam's aquaculture production from 2013 to 2018. The aquaculture in Vietnam includes: farmed fish production, farmed shrimp production and other aquatic products. Aquaculture production is divided by province and city.

In [5]:
# import the Vietnam map by provinces data
vietnam_geo = json.load(open("geodata/aquaculture_production_of_vietnam.geojson", "r"))

# Convert map properties/
state_id_map = {}
for feature in vietnam_geo["features"]:
    feature["id"] = feature["properties"]["Code"]
    state_id_map[feature["properties"]["Name"]] = feature["id"]

# Import aquaculture_production csv
df = pd.read_csv("geodata/tsnt_2013__2018_en.csv")
df.rename(columns={"Provincial code":'Code',"Provincial name":'Name'},inplace=True)
years = ['2013','2014','2015','2016','2017','2018']
for i, y in enumerate(years):
    scale = 'ProdScale'+ y
    prod = 'Production'+y
    df[scale] = np.log10(df[y])
    df[prod] = df[y]

# Convert wide to long format
tsnt = df.drop(years, axis=1)
final_tsnt = pd.wide_to_long(tsnt, stubnames=['ProdScale','Production'], i=['Name','Code'], j="year")
final_tsnt.reset_index(inplace=True)

### 1.2 Choropleth map using GeoJSON <a id="12"></a>

In [5]:
input_year = '2018'
fig = px.choropleth_mapbox(
    df,
    locations ="Code",
    geojson = vietnam_geo,
    color = "ProdScale" + input_year,
    hover_name = "Name",
    hover_data = [input_year],
    mapbox_style="carto-positron",
    center={"lat": 17, "lon": 106},
    zoom=4.5,
    title ="The Aquaculture Production of Vietnam by Provinces in " +input_year
)
fig.update_geos(fitbounds ="locations", visible=False)
fig.show()

<img src="figures/Aqua_prod_2013.png" alt="drawing" style="width:950px;"/>

### 1.3 Animated figures with GeoJSON, Plotly Express <a id="12"></a>

In [9]:
fig = px.choropleth(
    final_tsnt,
    locations ="Code",
    animation_frame = "year",
    geojson = vietnam_geo,
    color = "ProdScale",
    hover_name = "Name",
    hover_data = ["Production"],
    title ="The Aquaculture Production of Vietnam from 2013 to 2018 by Provinces",
)
fig.update_geos(fitbounds = "locations", visible=False)
fig.show()

## 2. Proportion of poor households by region in Vietnam from 1998 to 2016  <a id="2"></a>
The dataset includes le pourcentage of poor households by region in Vietnam from 1998 to 2016. The standard of poor households for this period based on the average income per person per month of households is updated according to the consumer price index as follows: In 2010, VND 400,000 for rural areas and VND 500,000 for urban areas; Similarly, in 2013 it was VND 570,000 and VND 710,000; in 2014, VND 605,000 dong and VND 750,000; in 2015, there were VND 615,000 and VND 760,000 dong; In 2016, VND 630,000 and VND 780,000 respectively.

In [ ]:
import geopandas as gpd

In [7]:
# import the Vietnam map by region data (error geojson file)
vnregion_geo = json.load(open("geodata/poverty_rate_1998_2016.geojson", "r",encoding='utf8'))

In [48]:
# Convert map properties
region_id_map = {}
for feature in vnregion_geo["features"]:
    feature["row"] = feature["properties"]["row"]
    region_id_map[feature["properties"]["Name_EN"]] = feature["row"]

# Import aquaculture_production csv
poverty_rate_1998_2016 = pd.read_csv("geodata/poverty_rate_1998_2016.csv",delimiter="\t")

cols = sorted(poverty_rate_1998_2016.columns[3:], reverse=False)

for i, y in enumerate(cols):
    scale = 'Poverty_percentage'+ y
    poverty = "Poverty" + y
    poverty_rate_1998_2016[scale] = poverty_rate_1998_2016[y]/100
    poverty_rate_1998_2016[poverty] = poverty_rate_1998_2016[y]

# Convert wide to long format
poverty = poverty_rate_1998_2016.drop(cols, axis=1)
final_poverty = pd.wide_to_long(poverty,['Poverty_percentage',"Poverty"], i=['Name_EN','Name_VI','row'], j= "year")
final_poverty.reset_index(inplace=True)


### 1.2 Choropleth map using GeoJSON <a id="21"></a>

In [51]:
input_year ='2013' # n1998	n2002	n2004	n2006	n2008	n2010	2011 n2012	n2013	n2014	n2015	n2016
fig = px.choropleth_mapbox(
    poverty_rate_1998_2016,
    locations = 'Name_EN',
    geojson = vnregion_geo,
    color = "Poverty_percentage"+ input_year ,
    hover_name = "Name_VI",
    hover_data = ["Poverty"+input_year],
    mapbox_style="carto-positron",
    center={"lat": 17,"lon": 106},
    zoom=4,
    title ="Proportion of poor households by region in Vietnam "+ input_year,
)
fig.update_geos(fitbounds = "locations", visible=False)
fig.show()

### 2.3 Animated figures with GeoJSON, Plotly Express <a id="22"></a>

In [16]:
fig = px.choropleth(
    final_poverty,
    locations = 'row',
    animation_frame = "year",
    geojson = vnregion_geo,
    color = "Poverty_percentage",
    hover_name = "Name_EN",
    hover_data = ['Poverty'],
    title = "Proportion of poor households by region in Vietnam from 1998 to 2016",
)
fig.update_geos(fitbounds = "locations", visible=False)
fig.show('browser')

## 3. The percentage of malnourished children under 5 in 2018 by locality

The attributes include the total weight, hight and weight based on height.

In [9]:
# Malnutrition data
malnutrition_children_vn_2018  = pd.read_csv("geodata/malnutrition_children_vn_2018.csv")

In [6]:
fig = px.choropleth_mapbox(
    malnutrition_children_vn_2018,
    locations = 'Code',
    geojson = vietnam_geo,
    color = 'Wei_Hei',
    hover_name = "Name",
    hover_data = ['Wei_Hei'],
    mapbox_style="carto-positron",
    center={"lat": 17,"lon": 106},
    zoom=4,
    title ="malnourished children under 5 in 2018 by locality in Vietnam ",
)
fig.update_geos(fitbounds = "locations", visible=False)
fig.show()

<img src="figures/Malnutrion_children_2018.png" alt="drawing" style="width:950px;"/>

## Important Notes <a id="333"></a>
Defining redraw: Setting redraw: false is an optimization for scatter plots so that animate just makes changes without redrawing the whole plot. For other plot types, such as contour plots, every frame must be a total plot redraw, i.e. redraw: true.

## Reference <a id="666"></a>
For additional information and attributes for creating bubble charts in Plotly see: https://plotly.com/python/bubble-charts/. 
For more documentation on creating animations with Plotly, see https://plotly.com/python/#animations.
https://plotly.com/python/animations/